In [112]:
from langchain_docling import DoclingLoader
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [23]:
FILE_PATH = "https://www.sintef.no/projectweb/mrst/publications/papers-by-mrst-team/"
loader = DoclingLoader(file_path=FILE_PATH)
docs = loader.load()

In [29]:
print(docs[0].metadata.get('dl_meta').get('doc_items'))

[{'self_ref': '#/texts/2', 'parent': {'$ref': '#/groups/2'}, 'children': [], 'content_layer': 'body', 'label': 'list_item', 'prov': []}, {'self_ref': '#/texts/3', 'parent': {'$ref': '#/groups/2'}, 'children': [], 'content_layer': 'body', 'label': 'list_item', 'prov': []}, {'self_ref': '#/texts/4', 'parent': {'$ref': '#/groups/2'}, 'children': [], 'content_layer': 'body', 'label': 'list_item', 'prov': []}, {'self_ref': '#/texts/5', 'parent': {'$ref': '#/groups/2'}, 'children': [], 'content_layer': 'body', 'label': 'list_item', 'prov': []}, {'self_ref': '#/texts/6', 'parent': {'$ref': '#/groups/2'}, 'children': [], 'content_layer': 'body', 'label': 'list_item', 'prov': []}, {'self_ref': '#/texts/7', 'parent': {'$ref': '#/groups/2'}, 'children': [], 'content_layer': 'body', 'label': 'list_item', 'prov': []}, {'self_ref': '#/texts/8', 'parent': {'$ref': '#/groups/2'}, 'children': [], 'content_layer': 'body', 'label': 'list_item', 'prov': []}, {'self_ref': '#/texts/9', 'parent': {'$ref': '#

In [64]:
response = requests.get(FILE_PATH)
soup = BeautifulSoup(response.content, 'html.parser')

documents = []

paper_lists = soup.find_all('div', class_ = "rich-text-field")[1]
for article in paper_lists.find_all("li"):
    links = []
    text = article.get_text(strip = True)
    for link in article.find_all('a'):
        links.append(link.get('href'))
    df ={"text": text, "links": links}
    documents.append(df)

In [71]:
FILE_PATH = "https://www.cambridge.org/core/services/aop-cambridge-core/content/view/7AC2425C73F6F729DB88DB1A504FA1E7/9781316519967AR.pdf/Advanced_Modeling_with_the_MATLAB_Reservoir_Simulation_Toolbox.pdf?event-type=FTLA"
loader = DoclingLoader(file_path=FILE_PATH)
docs = loader.load()

Token indices sequence length is longer than the specified maximum sequence length for this model (930 > 512). Running this sequence through the model will result in indexing errors


In [212]:
advanced_section_to_author = {
    1: ["Runar L. Berge", "Øystein S. Klemetsdal", "Knut-Andreas Lie"],
    2: ["Mohammed Al Kobaisi", "Wenjuan Zhang"],
    3: ["Øystein S. Klemetsdal", "Knut-Andreas Lie"],
    4: ["Knut-Andreas Lie", "Olav Møyner"],
    5: ["Olav Møyner"],
    6: ["Olav Møyner"],
    7: ["Xin Sun", "Knut-Andreas Lie", "Kai Bao"],
    8: ["Olav Møyner"],
    9: ["Daniel Wong", "Florian Doster", "Sebastian Geiger"],
    10: ["Olufemi Olorode", "Bin Wang", "Harun Ur Rashid"],
    11: ["Rafael March", "Christine Maier", "Florian Doster", "Sebastian Geiger"],
    12: ["Marine Collignon", "Øystein S. Klemetsdal", "Olav Møyner"],
    13: ["Jhabriel Varela", "Sarah E. Gasda", "Eirik Keilegavlen", "Jan Martin"],
    14: ["Odd Andersen"]
}

introduction_section_to_author = {
    1: ["Knut-Andreas Lie"],
    2: ["Knut-Andreas Lie"],
    3: ["Knut-Andreas Lie"],
    4: ["Knut-Andreas Lie"],
    5: ["Knut-Andreas Lie"],
    6: ["Knut-Andreas Lie"],
    7: ["Knut-Andreas Lie"],
    8: ["Knut-Andreas Lie"],
    9: ["Knut-Andreas Lie"],
    10: ["Knut-Andreas Lie"],
    11: ["Knut-Andreas Lie"],
    12: ["Knut-Andreas Lie"],
    13: ["Knut-Andreas Lie"],
    14: ["Knut-Andreas Lie"],
    15: ["Knut-Andreas Lie"],
}

def get_number(title):
    n_chars = len(title)
    number = ""
    n_dots = 0
    for i in range(n_chars):
        c = title[i]
        if c.isdigit() or c==".":
            number+=c
            if c==".":
                n_dots += 1
        else:
            rest_of_title = title[i:]
            return number, n_dots, rest_of_title
    return number, n_dots, title
    

def sort_book(book_df, name_df, metadata_df, file_name):
    numbers = book_df.keys()
    content = book_df.values()
    section_name = name_df.values()
    metadata = metadata_df.values()
    df = pd.DataFrame()
    numbers = [[int(i) for i in num.split(".")] for num in numbers]
    numbers = np.array(numbers)
    for i in range(3):
        df[str(i)] = numbers[:,i]

    if file_name == "Advanced Book":
        section_to_author_dict = advanced_section_to_author
    if file_name == "Introduction Book":
        section_to_author_dict = introduction_section_to_author

    df['authors'] = df['0'].apply(lambda x: section_to_author_dict.get(x))
    df['content'] = content
    df['section_name'] = section_name
    df['metadata'] = metadata
    df['file_type'] = file_name
    return df.sort_values(by = ['0','1','2'])

def generate_df(book_documents, file_name):
    sections = set()
    book_df = {}
    name_df = {}
    metadata_df = {}

    max_dots = 2

    for doc in book_documents:
        pg_content = doc.page_content
        split = pg_content.split("\n")
        title = split[0]
        content = "\n".join(split[1:])
        number, n_dots, name_title = get_number(title)
        if number != "" and int(number.split(".")[0]) <= 15:

            number += ".0"*(max_dots-n_dots)

            if number not in sections:
                sections.add(number)
                book_df[number] = content
                name_df[number] = name_title
                metadata_df[number] = doc.metadata
            else:
                book_df[number] += content

    advanced_book_df = sort_book(book_df, name_df, metadata_df, file_name)
    return advanced_book_df

In [213]:
advanced_book_df = generate_df(book_documents = docs, file_name = "Advanced Book")

In [217]:
advanced_book_df.to_pickle("advanced_book_df.pkl")

In [190]:
FILE_PATH = "https://www.cambridge.org/core/services/aop-cambridge-core/content/view/F48C3D8C88A3F67E4D97D4E16970F894/9781108492430AR.pdf/An_Introduction_to_Reservoir_Simulation_Using_MATLAB_GNU_Octave.pdf?event-type=FTLA"
loader = DoclingLoader(file_path=FILE_PATH)
docs_v2 = loader.load()

Token indices sequence length is longer than the specified maximum sequence length for this model (639 > 512). Running this sequence through the model will result in indexing errors


In [215]:
introduction_book_df = generate_df(book_documents = docs_v2, file_name = "Introduction Book")

In [218]:
introduction_book_df.to_pickle("introduction_book_df.pkl")